# Pennylane: I. Introduction to Quantum Computing
## I.11. Multi-Qubit Systems

In [1]:
# preparation
import numpy as np
import pennylane as qml
import pandas as pd
import altair as alt

### Codercise I.11.1 - Preparing basis state

In PennyLane, qubits are indexed numerically from left to right. The leftmost qubit is located at the top of the circuit. 

For $|1001\rangle$, the first qubit '1' is represented by the index '0'. The whole quantum number (QNum) is equal to $|9\rangle$.

**Task**: Return the computational basis vector $|n\rangle$ (3-qubit device) based on an integer input.

**Solution**: 
- The function `np.binary_repr()` changes a base-10 integer to a binary number.
    - '6' --> '110' 
- It takes two parameters; the integer to be transformed, and the number of digits in the binary number to fill 0s.
    - '6' --> integer
    - '110' --> binary number
    - len('110') --> number of digits
- We cannot represent the QNum if the number of wires is not enough, i.e., lower than the number of digits.
    - '110' requires 3 wires or more
- We can check each digit and apply the Pauli-X operation if the digit is '1'.
    - Make |000> into |110> to represent '110', digit-by-digit

In [2]:
num_wires = 3
dev = qml.device("default.qubit", wires=num_wires)


@qml.qnode(dev)
def make_basis_state(basis_id):
    """Produce the 3-qubit basis state corresponding to |basis_id>.

    Note that the system starts in |000>.

    Args:
        basis_id (int): An integer value identifying the basis state to construct.

    Returns:
        np.array[complex]: The computational basis state |basis_id>.
    """

    ##################
    # YOUR CODE HERE #
    ##################

    # CREATE THE BASIS STATE
    binary_num = np.binary_repr(basis_id, num_wires) # fill 0s if necessary
    if num_wires < len(binary_num): # 'not enough wire' case
        return qml.state()
    for i in range(len(binary_num)):
        if binary_num[i] == '1': # for 1s in |___>
            qml.PauliX(wires=i) # change |0> to |1>
    return qml.state()


basis_id = 3
print(f"Output state = {make_basis_state(basis_id)}")

Output state = [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


### Codercise I.11.2 - Separable operations

**Task**: Create the state $|+1\rangle=|+\rangle\otimes|1\rangle$. Evaluate:
- the expectation value of Y
- the expectation value of Z

**Solution**: We have two qubits. We prepare one qubit to have the state $|+\rangle$, using the Hadamard.
The remaining qubit is changed into $|1\rangle$ with the Pauli-X function. For expectation values, we can use `qml.expval()` with respective Pauli operations.

In [3]:
# Creates a device with *two* qubits
dev = qml.device("default.qubit", wires=2)


@qml.qnode(dev)
def two_qubit_circuit():
    ##################
    # YOUR CODE HERE #
    ##################

    # PREPARE |+>|1>
    qml.Hadamard(0) # create |+>
    qml.PauliX(1) # create |1>
    # RETURN TWO EXPECTATION VALUES, Y ON FIRST QUBIT, Z ON SECOND QUBIT
    return qml.expval(qml.PauliY(0)), qml.expval(qml.PauliZ(1))


print(two_qubit_circuit())

(tensor(0., requires_grad=True), tensor(-1., requires_grad=True))


### Codercise I.11.3 - Expectation value of two-qubit observable

**Task**: Create the state $|1-\rangle=|1\rangle\otimes|-\rangle$. Measure the expectation value of *two-qubit observable* $Z\otimes X$.

**Solution**: Tensor product is applied when the symbol `@` is used.

In [4]:
dev = qml.device("default.qubit", wires=2)


@qml.qnode(dev)
def create_one_minus():
    ##################
    # YOUR CODE HERE #
    ##################

    # PREPARE |1>|->
    qml.PauliX(0) # create |1>
    qml.PauliX(1) # prepare |1> to create |->
    qml.Hadamard(1) # create |->
    # RETURN A SINGLE EXPECTATION VALUE Z \otimes X
    return qml.expval(qml.PauliZ(0) @ qml.PauliX(1))


print(create_one_minus())

0.9999999999999996


### Plotting Function
Before moving on, we need to implement the plotting function to use.

In [5]:
# define pennylane plotter function
def plotter(theta, ZI_results, IZ_results, ZZ_results, combined_results):
    """Plot the value of the output

    Args:
        theta (np.array[float]): Angles for the x axis.
        ZI_results (np.array[float]): ZI expectation values
        IZ_results (np.array[float]): IZ expectation values
        ZZ_results (np.array[float]): ZZ expectation values
        combined_results (np.array[float]): users guess for how ZI and IZ results combine
            to produce ZZ results.
    """

    df = pd.DataFrame(
        data=np.array([theta, ZI_results, IZ_results, ZZ_results, combined_results]).T,
        columns=["theta", "ZI", "IZ", "ZZ", "Your guess ZZ"],
    )

    plot = (
        alt.Chart(df)
        .transform_fold(["ZI", "IZ", "ZZ", "Your guess ZZ"], ["observable", "expval"])
        .mark_line()
        .encode(x="theta:Q", y="expval:Q", color="observable:N")
    )

    return plot

### Codercise I.11.4 - Double Trouble

**Task**: Implement the following circuit twice.
1. First Version
    - Measure the observables Z on the first qubit ($Z\otimes I$).
    - Measure Z on the second qubit ($I\otimes Z$).
2. Second Version
    - Measure the observable $Z\otimes Z$.

<img height="40%" width="40%" src="https://assets.cloud.pennylane.ai/codebook/circuit_i-11-4.svg"/>

Plot the results as a function of $\theta$ to test your hypothesis.

**Solution**: Tensor product is applied when the symbol `@` is used. Identity portion of observables can be simplified. For instance, `qml.Identity(0) @ qml.PauliZ(1)` is reduced to `qml.PauliZ(1)`. The hypothesis is that the element-wise multiplication of $Z\otimes I$ and $I\otimes Z$ will give us $Z\otimes Z$.

In [6]:
dev = qml.device("default.qubit", wires=2)


@qml.qnode(dev)
def circuit_1(theta):
    """Implement the circuit and measure Z I and I Z.

    Args:
        theta (float): a rotation angle.

    Returns:
        float, float: The expectation values of the observables Z I, and I Z
    """
    ##################
    # YOUR CODE HERE #
    ##################

    qml.RX(theta, wires=0)
    qml.RY(2*theta, wires=1)
    
    return qml.expval(qml.PauliZ(0) @ qml.Identity(1)), qml.expval(qml.Identity(0) @ qml.PauliZ(1))


@qml.qnode(dev)
def circuit_2(theta):
    """Implement the circuit and measure Z Z.

    Args:
        theta (float): a rotation angle.

    Returns:
        float: The expectation value of the observable Z Z
    """

    ##################
    # YOUR CODE HERE #
    ##################

    qml.RX(theta, wires=0)
    qml.RY(2*theta, wires=1)

    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))


def zi_iz_combination(ZI_results, IZ_results):
    """Implement a function that acts on the ZI and IZ results to
    produce the ZZ results. How do you think they should combine?

    Args:
        ZI_results (np.array[float]): Results from the expectation value of
            ZI in circuit_1.
        IZ_results (np.array[float]): Results from the expectation value of
            IZ in circuit_2.

    Returns:
        np.array[float]: A combination of ZI_results and IZ_results that
        produces results equivalent to measuring ZZ.
    """

    combined_results = np.zeros(len(ZI_results))

    ##################
    # YOUR CODE HERE #
    ##################

    combined_results = np.multiply(ZI_results, IZ_results)
    return combined_results


theta = np.linspace(0, 2 * np.pi, 100)

# Run circuit 1, and process the results
circuit_1_results = np.array([circuit_1(t) for t in theta])

ZI_results = circuit_1_results[:, 0]
IZ_results = circuit_1_results[:, 1]
combined_results = zi_iz_combination(ZI_results, IZ_results)

# Run circuit 2
ZZ_results = np.array([circuit_2(t) for t in theta])

# Plot your results
plot = plotter(theta, ZI_results, IZ_results, ZZ_results, combined_results)
plot

alt.Chart(...)

This notebook is done by `Myanmar Youths` for `Womanium Quantum + AI 2024` program.
- <a href="https://www.linkedin.com/in/la-wun-nannda-b047681b5/"><u>La Wun Nannda</u></a>
- <a href="https://www.linkedin.com/in/chit-zin-win-46a2a3263/"><u>Chit Zin Win</u></a>